In [ ]:
# Import libraries
import os
import subprocess

# Change current working directory
os.chdir('/home/moonmoon/AllVideosAlbertson/Scripts/VideoProcessing')

### EDIT CROP.SH BETWEEN EACH RUN TO REPRESENT ACCURATE DIMENSIONS

In [ ]:
# Run the script and capture its output
result = subprocess.run(['bash', 'Crop.sh'], capture_output = True, text = True)

# Print the script's output
print(result.stdout)

# Print any errors that occurred
print(result.stderr)

# Print the exit code (0 indicates success)
print(result.returncode)

### RUN ONCE

In [ ]:
# Run the script and capture its output
result = subprocess.run(['bash', 'Auto_FFMPEG.sh'], capture_output = True, text = True)

# Print the script's output
print(result.stdout)

# Print any errors that occurred
print(result.stderr)

# Print the exit code (0 indicates success)
print(result.returncode)

### Start Tracking

In [ ]:
#Adding .py file to path to enable execution of .py functions in .ipynb
#import sys
#sys.path.append('/home/moonmoon/AllVideosAlbertson/Scripts')

# Import libraries and .py files
import shutil
import psutil
import time
import gc
import re

import matplotlib.pyplot as plt
import pickle as pkl
import numpy as np
import cv2 as cv

from ultralytics import YOLO
from sort_try import *
from tqdm import tqdm


In [ ]:

# Load a pretrained YOLOv8n model
model = YOLO('/home/moonmoon/AllVideosAlbertson/FishDetector/FishDetector/FD30.pt')

# Set source to video folder
source = "/home/moonmoon/AllVideosAlbertson/Group"
# Change current working directory to source path
os.chdir(source)

# Store list of folders at source/ToTrack by pulling file name if file is a folder
listog = [file.name for file in os.scandir(f'{source}/ToTrack') if file.is_dir()]

# Store a list of folders at source/ToTrack if file is a folder, split the result at the first underscore
list_sort = [(file.name).split('_', 1) for file in os.scandir(f'{source}/ToTrack') if file.is_dir()]

# Pull out the second item for each name tuple
Second_Column = [row[1] for row in list_sort]

# Convert all strings to uppercase, to standardize capitalization
list_sort = [x.upper() for x in Second_Column]

# Create empty dictionary to hold original name and sorted name together
list_crop = {}

# Loop through processed name, execute following commands
for i, j in enumerate(list_sort):
    # Assign processed name as key to original name
    list_crop[f'{j}'] = listog[i]

# Sort dictionary key, item pairs by alphabet
sorted(list_crop.items())

# Loop through all folders at source
for sortedfolder, ogfolder in list_crop.items():
    print(f'{ogfolder} to {sortedfolder}')

    # Reset KalmanBoxTracker
    KalmanBoxTracker.count = 0

    # Store list of files at source
    list_files = os.listdir(f'/home/moonmoon/AllVideosAlbertson/Group/ToTrack/{ogfolder}')
    #list_sort = [(file.name).split('_', maxsplit = 1) for file in os.scandir(source) if file.is_dir()]
    # Replace all non-digit characters with 1, sort by numeric value of original digits
    list_files.sort(key = lambda f: int(re.sub('\\D', '1', f)))

    # Set animal number
    animal_number = 1
    nframe = 0

    #create instance of SORT
    tracker = Sort(animal_number = animal_number, max_age = 18000, min_hits = 3, iou_threshold = 0.45)

    for avi in list_files:
        print(avi)
        if avi.endswith(".avi"):
            # Set names for various output files and global folders
            name, ext = os.path.splitext(avi)
            trackedname = f'tracked_{name}'
            cut, ogname = name.split('-', maxsplit = 1)
            output_path = f'{source}/Tracked/{ogname}'
            print(output_path)
            # If not output_path exists
            if not os.path.exists(output_path):
                os.mkdir(output_path) # make output_path

            # Open the video file
            cap = cv.VideoCapture(f'ToTrack/{ogfolder}/{avi}')

            # Get video file frame count
            frame_count = int(cap.get(cv.CAP_PROP_FRAME_COUNT))

            # Pull video width and height
            capw = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
            caph = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

            # Pull video fps
            fps = int(cap.get(cv.CAP_PROP_FPS))

            # Open video writer
            writer = cv.VideoWriter(f'{output_path}/{trackedname}.avi', cv.VideoWriter_fourcc(*'MJPG'), fps, (capw, caph))

            # Create box and track list for saving detections (pre- and post-SORT)
            boxlist = np.zeros([1,6])
            tracklist = np.zeros([1,6])

            # Create progress bar object
            pbar = tqdm(total = frame_count)

            # Loop through the video frames
            while cap.isOpened():
                #print('success')
                # Read a frame from the video
                success, frame = cap.read()

                psutil.cpu_percent()
                dict(psutil.virtual_memory()._asdict())
                psutil.virtual_memory().percent

                # Check if frame is successfully read or not
                if success and (psutil.virtual_memory().available * 100 / psutil.virtual_memory().total) > 25:

                    # Run inference on the source
                    results = model(frame, conf = 0.5, iou = 0.45, save_conf = True, max_det = animal_number, verbose = False)  # generator of Results objects

                    # Process results generator
                    for result in results:
                        if(result.boxes != None): # Check if detections are present or not.
                            boxes = result.boxes.xyxy.cpu().numpy()  # Boxes object for bbox outputs
                            conf = result.boxes.conf.cpu().numpy()[...,None] # Confidence object for bbox outputs
                            boxes = np.hstack([boxes, conf]) # Horizontally stack confidence with boxes
                            temp = np.zeros([boxes.shape[0], 1])
                            temp.fill(nframe)
                            boxes = np.hstack([boxes, temp])
                            boxlist = np.vstack([boxlist, boxes]) # Vertically stack confidence with boxes onto previous detections
                        elif(result.boxes == None): # Check if detections are present or not.
                            zerodetections = np.zeros([1,6]) # Generate line of zeros so as to better inform tracking of missed detections
                            boxlist = np.vstack([boxlist, zerodetections]) # Vertically stack onto previous detections

                    try:
                        # update SORT
                        nptrack = tracker.update(boxes)
                        temp = np.zeros([nptrack.shape[0], 1])
                        temp.fill(nframe)
                        nptrack = np.hstack([nptrack, temp])
                        tracklist = np.vstack([tracklist, nptrack])
                    except Exception as e:
                        # Release the video capture and writer objects and close the display window
                        cap.release()
                        writer.release()
                        cv.destroyAllWindows()
                        avisource = f'{source}/ToTrack/{ogfolder}/{avi}'
                        destination = source + "/Problems"
                        shutil.move(avisource, destination)
                        break

                    annotated_frame = frame
                    for track in nptrack:
                        # Pull x1, y1, x2, y2, id
                        x1, y1, x2, y2, id, nf = track
                        x1, y1, x2, y2, id, nf = int(x1), int(y1), int(x2), int(y2), int(id), int(nf)
                        # Visualize the results on the frame
                        annotated_frame = cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
                        annotated_frame = cv.putText(annotated_frame, str(id), (x1, y1), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                        # Display the annotated frame
                        cv.imshow("YOLOv8 Tracking", annotated_frame)

                    # Write frames
                    writer.write(annotated_frame)
                    nframe += 1 # Update frame count
                    pbar.update(1) # Update progress bar
                    # Break the loop if 'q' is pressed
                    if cv.waitKey(1) & 0xFF == ord("q"):
                        pbar.close()
                        break
                elif success and (psutil.virtual_memory().available * 100 / psutil.virtual_memory().total) < 25:
                    print(f'MEMORY ALERT! Saving progress...')
                    # Save the array
                    with open(f'{output_path}/detections-{trackedname}-{nframe}.pkl', 'wb') as f:
                        pkl.dump(boxlist, f)

                    # Save the array
                    with open(f'{output_path}/tracks-{trackedname}-{nframe}.pkl', 'wb') as f:
                        pkl.dump(tracklist, f)

                    print(f'Cleaning up garbage.')
                    gc.collect()

                    del boxlist, tracklist

                    boxlist = np.zeros([1,6])
                    tracklist = np.zeros([1,6])


                    while(psutil.virtual_memory().available * 100 / psutil.virtual_memory().total) < 35:
                        print(f'Taking a little break <3')
                        print(f'Sleeping...mem at {psutil.virtual_memory().available * 100 / psutil.virtual_memory().total}')
                        time.sleep(1)

                    print(f'Awake!')
                else:
                    # Break the loop if the end of the video is reached
                    pbar.close()
                    break
            pbar.close() # Close progress bar

            # Release the video capture and writer objects and close the display window
            cap.release()
            writer.release()
            cv.destroyAllWindows()

            avidestination = f'{source}/Originals/{ogname}'
            # If avidestination exists
            if not os.path.exists(avidestination):
                os.mkdir(avidestination) # make output_path

            # Move video cut to Cut subfolder named after ogname (source, destination)
            shutil.move(f'{source}/ToTrack/{ogfolder}/{avi}', avidestination)

            # Save the array
            with open(f'{output_path}/detections-{trackedname}-{nframe}.pkl', 'wb') as f:
                pkl.dump(boxlist, f)

            # Save the array
            with open(f'{output_path}/tracks-{trackedname}-{nframe}.pkl', 'wb') as f:
                pkl.dump(tracklist, f)

            gc.collect()

cropCichlid1_080624_2_ind1 to 080624_2_IND1
cropCichlid1_011724_1_ind2 to 011724_1_IND2
cut6-cropCichlid1_011724_1_ind2.avi
/home/moonmoon/AllVideosAlbertson/Group/Tracked/cropCichlid1_011724_1_ind2


  0%|          | 0/216000 [00:00<?, ?it/s]

auto Assigned id


100%|█████████▉| 215999/216000 [41:01<00:00, 87.75it/s] 


cropCichlid1_110123_2_ind2 to 110123_2_IND2
cut1-cropCichlid1_110123_2_ind2.avi
/home/moonmoon/AllVideosAlbertson/Group/Tracked/cropCichlid1_110123_2_ind2


...

100%|██████████| 216000/216000 [39:49<00:00, 90.38it/s] 


cut2-cropCichlid1_022124_1_ind1.avi
/home/moonmoon/AllVideosAlbertson/Group/Tracked/cropCichlid1_022124_1_ind1


100%|█████████▉| 215773/216000 [38:53<00:02, 92.47it/s] 


cut3-cropCichlid1_022124_1_ind1.avi
/home/moonmoon/AllVideosAlbertson/Group/Tracked/cropCichlid1_022124_1_ind1


 34%|███▍      | 73889/216000 [12:24<24:15, 97.61it/s] 